In [7]:
# ========================================
# Cell 1: 라이브러리 import 및 설정
# ========================================

import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('../src'))

from processors.pdf_text_extractor import PDFTextExtractor
from processors.pdf_pair_matcher import PDFPairMatcher
from processors.translation_pair_generator import TranslationPairGenerator
from config import PROCESSED_DIR, PDF_DOWNLOAD_DIR

# 처리할 언어 쌍들 정의
LANGUAGE_PAIRS = [
    ('en', 'ko'),  # 영어-한국어
    ('en', 'ja'),  # 영어-일본어
]

# 필요한 디렉토리 생성
os.makedirs(PROCESSED_DIR, exist_ok=True)

print(f"🌐 처리할 언어 쌍: {LANGUAGE_PAIRS}")
print(f"📁 PDF 폴더: {PDF_DOWNLOAD_DIR}")
print(f"📁 결과 저장 폴더: {PROCESSED_DIR}")

# PDF 폴더 존재 확인
for lang_pair in LANGUAGE_PAIRS:
    for lang in lang_pair:
        lang_dir = os.path.join(PDF_DOWNLOAD_DIR, lang)
        if os.path.exists(lang_dir):
            pdf_count = len([f for f in os.listdir(lang_dir) if f.endswith('.pdf')])
            print(f"   - {lang.upper()}: {pdf_count}개 PDF")
        else:
            print(f"   - {lang.upper()}: ❌ 폴더 없음 ({lang_dir})")




🌐 처리할 언어 쌍: [('en', 'ko'), ('en', 'ja')]
📁 PDF 폴더: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/2_downloaded/pdfs
📁 결과 저장 폴더: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed
   - EN: 101개 PDF
   - KO: ❌ 폴더 없음 (/mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/2_downloaded/pdfs/ko)
   - EN: 101개 PDF
   - JA: 95개 PDF


In [8]:
# ========================================
# Cell 2: 모든 언어 쌍 처리
# ========================================

extractor = PDFTextExtractor()
generator = TranslationPairGenerator(similarity_threshold=0.9)

all_results = {}

for source_lang, target_lang in LANGUAGE_PAIRS:
    print(f"\n{'='*80}")
    print(f"🔄 {source_lang.upper()}-{target_lang.upper()} 언어 쌍 처리 시작")
    print(f"{'='*80}")
    
    # 1. PDF 페어 찾기
    matcher = PDFPairMatcher(source_lang, target_lang)
    pdf_pairs = matcher.find_pdf_pairs()
    
    if not pdf_pairs:
        print(f"❌ {source_lang.upper()}-{target_lang.upper()} PDF 페어를 찾을 수 없습니다.")
        continue
    
    # 2. 번역 쌍 생성
    pairs_for_this_language = []
    
    for i, (source_path, target_path) in enumerate(pdf_pairs):
        print(f"\n[{i+1}/{len(pdf_pairs)}] 처리 중: {os.path.basename(target_path)}")
        
        try:
            # 텍스트 추출
            source_segments = extractor.extract_segments_from_pdf(source_path)
            target_segments = extractor.extract_segments_from_pdf(target_path)
            
            print(f"   - {source_lang.upper()}: {len(source_segments)}개, {target_lang.upper()}: {len(target_segments)}개 세그먼트")
            
            # 번역 쌍 생성
            pairs = generator.generate_pairs(
                source_segments, target_segments, target_path, 
                source_lang, target_lang
            )
            pairs_for_this_language.extend(pairs)
            
            print(f"   - ✅ {len(pairs)}개 번역 쌍 생성")
            
        except Exception as e:
            print(f"   - ❌ 오류: {e}")
    
    all_results[f"{source_lang}_{target_lang}"] = pairs_for_this_language
    print(f"\n🎉 {source_lang.upper()}-{target_lang.upper()}: 총 {len(pairs_for_this_language)}개 번역 쌍 완료!")





🔄 EN-KO 언어 쌍 처리 시작
❌ 매칭 테이블을 찾을 수 없습니다: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/1_input/pdf_list_master_en_ko.csv
❌ EN-KO PDF 페어를 찾을 수 없습니다.

🔄 EN-JA 언어 쌍 처리 시작
📋 EN-JA 매칭 테이블에서 101개 항목을 로드했습니다.
✅ 실제 존재하는 100개의 EN-JA PDF 페어를 발견했습니다.

[1/100] 처리 중: nextseq-1000-2000-wgs-app-note-M-GL-01148-jpn.pdf
   - EN: 163개, JA: 108개 세그먼트
🤖 임베딩 모델을 로드하는 중...
✅ 임베딩 모델 로드 완료.


/home/cjkim/anaconda3/envs/mywork/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   - ✅ 19개 번역 쌍 생성

[2/100] 처리 중: trusight-oncology-500-ctdna-data-sheet-m-gl-00843-jpn.pdf
   - EN: 356개, JA: 176개 세그먼트


/home/cjkim/anaconda3/envs/mywork/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   - ✅ 63개 번역 쌍 생성

[3/100] 처리 중: scrna-seq-app-note-m-gl-01139-jpn.pdf
   - EN: 1026개, JA: 583개 세그먼트


...

   - EN: 148개, JA: 68개 세그먼트


/home/cjkim/anaconda3/envs/mywork/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   - ✅ 30개 번역 쌍 생성

[100/100] 처리 중: instrument-services-training-brochure-m-gl-02786-jpn.pdf
   - EN: 250개, JA: 43개 세그먼트


/home/cjkim/anaconda3/envs/mywork/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   - ✅ 14개 번역 쌍 생성

🎉 EN-JA: 총 9682개 번역 쌍 완료!


In [9]:
# ========================================
# Cell 3: 결과 저장
# ========================================

total_pairs = 0

for lang_pair, pairs in all_results.items():
    if pairs:
        # 언어별 개별 파일 저장
        csv_path = os.path.join(PROCESSED_DIR, f"pdf_translation_pairs_{lang_pair}.csv")
        df = pd.DataFrame(pairs)
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        
        total_pairs += len(pairs)
        print(f"💾 {lang_pair.upper()} 저장: {csv_path} ({len(pairs)}개)")
        print(f"   📊 평균 유사도: {df['similarity_score'].mean():.3f}")

# 통합 파일 저장 (옵션)
if total_pairs > 0:
    all_pairs_combined = []
    for pairs in all_results.values():
        all_pairs_combined.extend(pairs)
    
    combined_csv_path = os.path.join(PROCESSED_DIR, "pdf_translation_pairs_all.csv")
    combined_df = pd.DataFrame(all_pairs_combined)
    combined_df.to_csv(combined_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"\n📦 통합 파일 저장: {combined_csv_path}")
    print(f"   📊 전체 번역 쌍: {len(all_pairs_combined)}개")
    print(f"   🌐 지원 언어: {combined_df['target_lang'].unique().tolist()}")




💾 EN_JA 저장: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed/pdf_translation_pairs_en_ja.csv (9682개)
   📊 평균 유사도: 0.965

📦 통합 파일 저장: /mnt/d/Cloud-Synced/Illumina/OneDrive - Illumina, Inc/aem_qa_system/data/3_processed/pdf_translation_pairs_all.csv
   📊 전체 번역 쌍: 9682개
   🌐 지원 언어: ['ja']


In [ ]:
# ========================================
# Cell 4: 결과 요약
# ========================================

if total_pairs > 0:
    print(f"\n{'='*80}")
    print("📈 다국어 PDF 번역 쌍 추출 완료!")
    print(f"{'='*80}")
    
    for lang_pair, pairs in all_results.items():
        if pairs:
            source_lang, target_lang = lang_pair.split('_')
            df = pd.DataFrame(pairs)
            print(f"\n🌐 {source_lang.upper()}-{target_lang.upper()}:")
            print(f"   📄 번역 쌍: {len(pairs)}개")
            print(f"   📊 평균 유사도: {df['similarity_score'].mean():.3f}")
            print(f"   📁 소스 파일: {df['source_file'].nunique()}개")
            
            # 상위 1개 샘플 보기
            if len(pairs) > 0:
                top_pair = df.nlargest(1, 'similarity_score').iloc[0]
                print(f"   🔍 최고 유사도 ({top_pair['similarity_score']:.3f}):")
                print(f"     {source_lang.upper()}: {top_pair['source_text'][:50]}...")
                print(f"     {target_lang.upper()}: {top_pair['target_text'][:50]}...")
else:
    print("\n⚠️ 생성된 번역 쌍이 없습니다.")